# 제주 핫 플레이스 지도에 표시하기

### 1. 데이터 준비

In [1]:
from google.colab import files
uploaded = files.upload()
filename = list(uploaded.keys())[0]

Saving 제주핫플레이스.csv to 제주핫플레이스.csv


In [2]:
import pandas as pd
raw_df = pd.read_csv(filename)
raw_df.head()

,content,date,like,place,tags
0,더 늦기전에 제주도로 떠나쟈😍.#제주핫플레이스 간단히 1탄 모아봄!#제주여행 요 필...,2018-12-07,"1,402",NaN,"['#제주핫플레이스', '#제주여행', '#제주여행', '#제주도여행', '#제주가..."
1,12월에 제주도를 가야하는이유🍊.#제주핫플 모음 끝판왕😍이거 하나면 겨울 #제주여행...,2018-12-03,368,NaN,"['#제주핫플', '#제주여행', '#제주', '#제주도', '#제주도맛집', '#..."
2,11월 놓치지 말아야 할 제주 관광.가을바람이 최고조에 이르는 11월추운 겨울을 앞...,2018-11-02,166,NaN,"['#honestin', '#어니스틴', '#제주여행', '#제주', '#제주도',..."
3,국민학교세대#제주관광#제주살이#제주이주민#아라동주민#삼남매집 #새해첫날#드라이브#명...,2019-01-01,28,명월국민학교,"['#제주관광', '#제주살이', '#제주이주민', '#아라동주민', '#삼남매집'..."
4,#제주관광 #제주 #돔나이트 #스트레스 #풀자 #춤추며 #땀날려 #가끔은괜찮아 #인...,2019-01-01,12,NaN,"['#제주관광', '#제주', '#돔나이트', '#스트레스', '#풀자', '#춤추..."


In [3]:
location_counts = raw_df.place.value_counts().to_frame()
print(location_counts.shape)
location_counts.head()

(1027, 1)


,place
Jeju,241
Jeju-do,172
Jungle Book by Alice,101
Seogwipo,65
제주도 크리스마스 박물관,59


In [4]:
location_counts.tail()

,place
성수족발,1
Uncommon Table,1
오렌지블루스,1
바다이야기,1
홍키호테,1


In [5]:
location_counts.to_csv('location_counts.csv')

In [8]:
loc_df = pd.read_csv('location_counts.csv')
loc_df.columns = ['지명','빈도']
loc_df.head()

,지명,빈도
0,Jeju,241
1,Jeju-do,172
2,Jungle Book by Alice,101
3,Seogwipo,65
4,제주도 크리스마스 박물관,59


### 2. 카카오 검색 API를 이용한 장소 검색
- 키워드 장소 검색

In [10]:
uploaded = files.upload()
filename = list(uploaded.keys())[0]

Saving kakaoapikey.txt to kakaoapikey.txt


In [11]:
with open(filename) as f:
    api_key = f.read()

In [12]:
import requests
from urllib.parse import quote

In [14]:
keyword = '합정 스타벅스'
search_url = 'https://dapi.kakao.com/v2/local/search/keyword.json'
url = f'{search_url}?query={quote(keyword)}'
# Authorization: KakaoAK {REST_API_KEY}
result = requests.get(url,
            headers={"Authorization": f'KakaoAK {api_key}'}).json()
result['documents'][0]

{'address_name': '서울 마포구 서교동 395-166',
 'category_group_code': 'CE7',
 'category_group_name': '카페',
 'category_name': '음식점 > 카페 > 커피전문점 > 스타벅스',
 'distance': '',
 'id': '26572121',
 'phone': '1522-3232',
 'place_name': '스타벅스 서교점',
 'place_url': 'http://place.map.kakao.com/26572121',
 'road_address_name': '서울 마포구 양화로 78',
 'x': '126.916980454434',
 'y': '37.5514601750423'}

In [20]:
import numpy as np

In [22]:
def find_place(keyword):
    url = f'{search_url}?query={quote(keyword)}'
    result = requests.get(url,
                headers={"Authorization": f'KakaoAK {api_key}'}).json()
    try:
        name = result['documents'][0]['place_name']
        lng = float(result['documents'][0]['x'])
        lat = float(result['documents'][0]['y'])
    except:
        print(keyword)
        name = ''
        lng = np.nan
        lat = np.nan

    return name, lng, lat, keyword

In [23]:
find_place('제주공항')

('제주국제공항', 126.492769004244, 33.5070789578184, '제주공항')

In [24]:
# loc_df에 공식명칭, 위도, 경도 정보 추가
name_list, lng_list, lat_list = [],[],[]
for place in loc_df.지명:
    name, lng, lat, _ = find_place(place)
    name_list.append(name)
    lng_list.append(lng)
    lat_list.append(lat)

Jeju-do
Jungle Book by Alice
성산일출봉 城山日出峰  Seongsan Ilchulbong
폼포코식당_pompokokitchen
Nimome
제주도 서귀포 중문관광단지 濟州島西歸浦中文觀光團地
제주고궁한복카페 jeju gogung hanbok studio
김녕미로공원 Jeju Kimnyoung Maze Park
Jeju Island 제주특별자치도 济州道
위미2리 동백군락지
한라산 (漢拏山, Hallasan)
제주 꿈꾸는고래 스쿠버&게스트하우스
牛岛 Udo Island 우도
고집돌우럭-gozipfish
제주 성산 신산포구자연산횟집 회포장
퍼시픽랜드 - Pacific Land JEJU
Casa Linda Guesthouse-까사 린다 펜션
the cliff jeju - 더클리프 제주
The Shilla Jeju
몽상드 애월 (Mônsant)
함덕반하다
Sunnyside Beachclub Jeju
성산일출봉 게스트하우스 파티 띵까게스트하우스
아날로그감귤체험
해발1950m 한라산 백록담
Busan, South Korea
새별오름 (Saebyul Orum)
천지연폭포 (Cheonjiyeon Waterfall)
Jeju horse Riding farm
Hamdeok Jeju
Seoul, South Korea
Guzasien -구좌지앵
Paris, France
환상의 섬 Jeju Island
Harmduck Beach in Jeju Island
스윗슬로cafe
커피파인더coffeefinder제주시청
Namwon Seogwipo Jeju,South korea
Meal Table Garden
쌀다방
Hamdoek Beach, Jeju Island
Hyatt Regency, Jeju Island
제주 서귀포 그릴 데미그라스
동굴카페 다희연(Cafe Daheeyeon in the Cave)
Ulsan, South Korea
Jeju_Gimnyeong_Maze_Park
Daegu, South Korea
Hotel Alfonso XIII, a Luxury Colle